## Quantum Machine Learning

## References

- The paper ['Circuit-centric quantum classifiers', by Maria Schuld, Alex Bocharov, Krysta Svore and Nathan Wiebe](https://arxiv.org/abs/1804.00633). Used it to understand the concept of quantum classifiers and the implementation of the quantum circuit for the same.
- [QunatumKatas/tutorials/QuantumClassification](https://github.com/microsoft/QuantumKatas/tree/main/tutorials/QuantumClassification) provided the basic structure of the code and the implementation of the quantum circuit for the quantum classifier.



## Running Environment

- Executed on docker container built using the Dockerfile in the QuantumKatas repository.
- It had few issues with iqsharp and proj deps. Fixed it. See [PR](https://github.com/microsoft/QuantumKatas/pull/900)

### Load train and test data

In [1]:
import json

# load data from json file
with open('train_test_split.json', 'r') as f:
    data = json.load(f)
    
X_train_list = data['train']['features']
y_train_list = data['train']['labels']

X_test_list = data['test']['features']
y_test_list = data['test']['labels']

In [2]:
## Generate Initial Weights

import random
# Set a seed for reproducibility
random.seed(42)

# Function to generate random numbers between 0 and 5
def generate_random_numbers():
    return random.uniform(0, 5)


initial_weights = []
for i in range(8):
    arr = []
    for i in range(8):
        arr.append(generate_random_numbers())
    initial_weights.append(arr)
    
initial_weights

[[3.1971339922894186,
  0.12505377611333468,
  1.3751465918455963,
  1.1160536907441139,
  3.682356070820062,
  3.3834974371145563,
  4.460897838524227,
  0.43469416314708076],
 [2.1096090984263522,
  0.14898609719035172,
  1.0931898740180168,
  2.526776440516812,
  0.13267984841931812,
  0.9941882534332425,
  3.249422188897616,
  2.724707403016083],
 [1.1022031102034835,
  2.946328419379544,
  4.047152283389133,
  0.032493798390305084,
  4.02909625916404,
  3.4906969749411343,
  1.7012525825899594,
  0.7773974990589078],
 [4.786065361033906,
  1.682972725563134,
  0.46372921690073954,
  0.48358188416732006,
  4.237471831737299,
  3.0186301568344556,
  4.035641366371901,
  3.6486589334690893],
 [2.6811404572735036,
  4.865578819896853,
  1.8926718860417675,
  2.760203156366135,
  4.1470233212649745,
  3.0925987618212303,
  4.308534501553886,
  2.88676072628381],
 [3.5228591810746175,
  0.22912191827831108,
  1.1394913782577343,
  1.4469398180105357,
  0.3989598846181375,
  1.1639544318

## Quantum Machine Learning

In [3]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
plt.style.use('ggplot')
import qsharp
qsharp.packages.add("Microsoft.Quantum.MachineLearning::0.28.302812")
qsharp.reload()


Preparing Q# environment...


Adding package Microsoft.Quantum.MachineLearning::0.28.302812.

Reloading workspace.

In [4]:
from Microsoft.Quantum.Samples import (
    TrainModel, ValidateModel, ClassifyModel
)

In [5]:
weights, bias = TrainModel.simulate(
    trainingVectors=X_train_list,
    trainingLabels=y_train_list,
    initialParameters=initial_weights,
    verbose=True
)

KeyboardInterrupt: 

## Validate the model

In [ ]:
miss_rate = ValidateModel.simulate(
    validationVectors=X_test_list,
    validationLabels=y_test_list,
    parameters=weights, bias=bias
)

print(f"Miss rate: {miss_rate:0.2%}")

## Classify

In [ ]:
actual_labels = y_test_list
classified_labels = ClassifyModel.simulate(
    samples=X_test_list,
    parameters=weights, bias=bias,
    tolerance=0.005, nMeasurements=10_000
)

In [ ]:
cases = [(0, 0), (0, 1), (1, 1), (1, 0)]
markers = [
    '.' if actual == classified else 'x'
    for (actual, classified) in cases
]
colormap = cmx.ScalarMappable(colors.Normalize(vmin=0, vmax=len(cases) - 1))
colors = [colormap.to_rgba(idx_case) for (idx_case, case) in enumerate(cases)]


In [ ]:

plt.figure(figsize=(12, 8))
for (idx_case, ((actual, classified), marker, color)) in enumerate(zip(cases, markers, colors)):
    mask = np.logical_and(np.equal(actual_labels, actual), np.equal(classified_labels, classified))
    if not np.any(mask): continue
    plt.scatter(
        X_test_list[mask, 0], X_test_list[mask, 1],
        label=f"Was {actual}, classified {classified}",
        marker=marker, s=300, c=[color],
    )
plt.legend()